In [3]:
import pandas as pd
import numpy as np
from datasets import load_dataset

In [ ]:
squad_dataset = load_dataset('squad')
print(squad_dataset['train'][0])

In [ ]:
import pandas as pd
df_correct = pd.DataFrame(squad_dataset['train'])
df_correct = df_correct[['question', 'answers']]
df_correct['Answer'] = df_correct['answers'].apply(lambda x: x['text'][0] if isinstance(x['text'], list) else x['text'])
df_correct['Label'] = 1
df_correct = df_correct.drop(columns=['answers'])
print(df_correct.head())



In [ ]:
df_faulty = pd.read_excel('Dataset_CSE584_Project.xlsx')

df_faulty['Label'] = 0
df = pd.concat([df_faulty, df_correct], ignore_index=True)

print(df.head())

In [ ]:
import pandas as pd


print("Data type of 'combined_text' column:", df['combined_text'].dtype)

non_string_rows = df[~df['combined_text'].apply(lambda x: isinstance(x, str))]
if not non_string_rows.empty:
    print("Rows with non-string values:")
    print(non_string_rows)

df['combined_text'] = df['combined_text'].astype(str)

print(f"NaN values in 'combined_text': {df['combined_text'].isna().sum()}")
print(f"Empty strings in 'combined_text': {(df['combined_text'] == '').sum()}")

df['text_length'] = df['combined_text'].apply(len)

print(df[['combined_text', 'text_length']].head())

df = df[df['text_length'] > 0]

df['combined_text'] = df['combined_text'].replace('', 'No valid content')

print(f"NaN values after replacement: {df['combined_text'].isna().sum()}")
print(f"Empty strings after replacement: {(df['combined_text'] == '').sum()}")



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['combined_text'])

y = df['Label']

df['text_length'] = df['combined_text'].apply(len)
print(df[['combined_text', 'text_length']].head())

df = df[df['text_length'] > 0]
print(X.shape)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])
